In [ ]:
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DATA_path      = '/content/drive/MyDrive/Sales_Dataset.csv'
DATA = pd.read_csv(DATA_path, engine="pyarrow")

Mounted at /content/drive


In [ ]:
DATA['Month'] = DATA['Year-Month'].str[5:7]
# monthly_profit = DATA.groupby('Month')['Profit'].sum().sort_index()
# monthly_profit.plot(kind='bar', figsize=(8, 5))
# plt.show()

In [ ]:
DATA['Order Date'] = pd.to_datetime(DATA['Order Date'])
DATA['day'] = DATA['Order Date'].dt.day
DATA['year'] = DATA['Order Date'].dt.year
# monthly_profit = DATA.groupby('day')['Profit'].sum().sort_index()
# monthly_profit.plot(kind='bar', figsize=(8, 5))
# plt.show()

In [ ]:
DATA = DATA[(np.abs(zscore(DATA[['Amount', 'Profit', 'Quantity']])) < 3).all(axis=1)]

In [ ]:
# DATA['Amount'] = DATA['Amount'] /DATA['Amount'].abs().max()
# DATA['Quantity'] = DATA['Quantity'] /DATA['Quantity'].abs().max()

In [ ]:
DATA = DATA.drop(['Order ID', 'Order Date','Year-Month','CustomerName'], axis=1)

In [ ]:
display(DATA)

,Amount,Profit,Quantity,Category,Sub-Category,PaymentMode,State,City,Month,day,year
0,9726,1275,5,Electronics,Electronic Games,UPI,Florida,Miami,06,27,2023
1,9726,1275,5,Electronics,Electronic Games,UPI,Illinois,Chicago,12,27,2024
2,9726,1275,5,Electronics,Electronic Games,UPI,New York,Buffalo,07,25,2021
3,4975,1330,14,Electronics,Printers,UPI,Florida,Miami,06,27,2023
4,4975,1330,14,Electronics,Printers,UPI,Illinois,Chicago,12,27,2024
...,...,...,...,...,...,...,...,...,...,...,...
1189,8825,3594,15,Furniture,Tables,Debit Card,New York,New York City,07,31,2024
1190,2082,642,8,Electronics,Phones,EMI,New York,Rochester,06,2,2020
1191,2082,642,8,Electronics,Phones,EMI,Texas,Austin,12,15,2022
1192,2082,642,8,Electronics,Phones,EMI,New York,Buffalo,08,7,2020


In [ ]:
categoricals = ['Category', 'Sub-Category', 'PaymentMode', 'State', 'City']

# DATA_enc = pd.get_dummies(DATA, columns=categoricals,
#                           drop_first=True,
#                           dtype=int)              # saves memory, no True/False

# print(DATA_enc.head())




# # 1. Original profit
# y = DATA_enc['Profit']

# # 2. Log-transform the target
# y_log = np.log1p(y)

# # 3. Split
# X = DATA_enc.drop('Profit', axis=1)


from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

for col in categoricals:
    DATA[col] = label_encoder.fit_transform(DATA[col])

display(DATA.head())

# 1. Original profit
y = DATA['Profit']

# 2. Log-transform the target
y_log = np.log1p(y)

# 3. Split
X = DATA.drop('Profit', axis=1)

,Amount,Profit,Quantity,Category,Sub-Category,PaymentMode,State,City,Month,day,year
0,9726,1275,5,0,3,4,1,9,06,27,2023
1,9726,1275,5,0,3,4,2,2,12,27,2024
2,9726,1275,5,0,3,4,3,1,07,25,2021
3,4975,1330,14,0,9,4,1,9,06,27,2023
4,4975,1330,14,0,9,4,2,2,12,27,2024


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


X_train, X_test, y_train_log, y_test_log = train_test_split(
    X, y_log, test_size=0.1, random_state=42
)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=300, max_depth=12, random_state=3)
rf.fit(X_train, y_train_log)

# Predict (log scale)
rf_preds_log = np.round(rf.predict(X_test), 5)

# Convert back to original scale
rf_preds = np.expm1(rf_preds_log)
y_test_actual = np.expm1(y_test_log)

# Compute MSE in normal scale
mse = np.mean(abs(y_test_actual - rf_preds))
mean_actual = np.mean(y_test_actual)
relative_mse = mse / mean_actual

print("🔍 Random Forest Results")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Average of actual values: {mean_actual:.2f}")
print(f"Relative MSE (MSE / mean actual): {relative_mse:.4f}\n")


random_integer = random.randint(0, len(X_test) - 1)

print(X_test.iloc[random_integer])
print()
print(y_test_actual.iloc[random_integer])
print(rf_preds[random_integer])

🔍 Random Forest Results
Mean Squared Error (MSE): 552.00
Average of actual values: 1283.71
Relative MSE (MSE / mean actual): 0.4300



NameError: name 'random' is not defined

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor

pipeline = Pipeline([
    ('scale', StandardScaler()),
    ('nn', MLPRegressor(hidden_layer_sizes=(32 , 16 , 16), max_iter=1200, alpha=0.001, random_state=2))
])

pipeline.fit(X_train, y_train_log)

# Predict and revert
mlp_preds_log = np.round(pipeline.predict(X_test), 5)
mlp_preds = np.expm1(mlp_preds_log)

# Compute error on original scale
#mse = mean_squared_error(y_test_actual, mlp_preds)

mse = np.mean(abs(y_test_actual - mlp_preds))
relative_mse = mse / mean_actual

print("\n🔍 MLP Results")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Average of actual values: {mean_actual:.2f}")
print(f"Relative MSE (MSE / mean actual): {relative_mse:.4f}\n")


print(X_test.iloc[random_integer])
print()
print(y_test_actual.iloc[random_integer])
print(mlp_preds[random_integer])


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor

import random

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPRegressor

rf_mae_list = []
mlp_mae_list = []

for i in range(10):
    X_train, X_test, y_train_log, y_test_log = train_test_split(
        X, y_log, test_size=0.1, random_state=i
    )

    # --- Random Forest ---
    rf = RandomForestRegressor(n_estimators=300, max_depth=12, random_state=i)
    rf.fit(X_train, y_train_log)
    rf_preds = np.expm1(rf.predict(X_test))
    y_test_actual = np.expm1(y_test_log)

    mae_rf = np.mean(abs(y_test_actual - rf_preds))
    mean_actual = np.mean(y_test_actual)
    relative_mae_rf = mae_rf / mean_actual
    rf_mae_list.append(mae_rf)

    print(f"RF Run {i+1} - MAE: {mae_rf:.2f}, Relative MAE: {relative_mae_rf:.4f}")

    # --- MLP ---
    pipeline = Pipeline([
        ('scale', StandardScaler()),
        ('nn', MLPRegressor(hidden_layer_sizes=(32 , 16 , 16), max_iter=1200, alpha=0.001, random_state=2))
    ])
    pipeline.fit(X_train, y_train_log)
    mlp_preds = np.expm1(pipeline.predict(X_test))

    mae_mlp = np.mean(abs(y_test_actual - mlp_preds))
    relative_mae_mlp = mae_mlp / mean_actual
    mlp_mae_list.append(mae_mlp)

    print(f"MLP Run {i+1} - MAE: {mae_mlp:.2f}, Relative MAE: {relative_mae_mlp:.4f}\n")

# --- Summary ---
print("Final Summary:")
print(f"Random Forest - Avg MAE: {np.mean(rf_mae_list):.2f}, Std Dev: {np.std(rf_mae_list):.2f}")
print(f"MLP           - Avg MAE: {np.mean(mlp_mae_list):.2f}, Std Dev: {np.std(mlp_mae_list):.2f}")


RF Run 1 - MAE: 598.95, Relative MAE: 0.4484
MLP Run 1 - MAE: 810.51, Relative MAE: 0.6068

RF Run 2 - MAE: 618.58, Relative MAE: 0.4804
MLP Run 2 - MAE: 815.49, Relative MAE: 0.6333

RF Run 3 - MAE: 710.63, Relative MAE: 0.4828
MLP Run 3 - MAE: 942.02, Relative MAE: 0.6400

RF Run 4 - MAE: 544.97, Relative MAE: 0.4052
MLP Run 4 - MAE: 847.95, Relative MAE: 0.6305

RF Run 5 - MAE: 598.38, Relative MAE: 0.4237
MLP Run 5 - MAE: 796.30, Relative MAE: 0.5638

RF Run 6 - MAE: 576.37, Relative MAE: 0.4738
MLP Run 6 - MAE: 810.52, Relative MAE: 0.6663

RF Run 7 - MAE: 616.38, Relative MAE: 0.4642
MLP Run 7 - MAE: 847.87, Relative MAE: 0.6386

RF Run 8 - MAE: 603.81, Relative MAE: 0.4760
MLP Run 8 - MAE: 930.47, Relative MAE: 0.7335

RF Run 9 - MAE: 627.11, Relative MAE: 0.4874
MLP Run 9 - MAE: 995.74, Relative MAE: 0.7739

RF Run 10 - MAE: 671.42, Relative MAE: 0.4368
MLP Run 10 - MAE: 823.50, Relative MAE: 0.5358

Final Summary:
Random Forest - Avg MAE: 616.66, Std Dev: 44.18
MLP           -